In [1]:
import pickle
import torch

In [115]:
def first_non_zero(x):
    x_value_mask = (x!=0).type(torch.int)
    idx = torch.arange(1, x.shape[-1] + 1).unsqueeze(0) # so that "0" is going to be a marker for lack of occurence
    idx = x_value_mask  * idx
    idx = torch.where(idx!=0, idx, float("inf")) - 1
    idx = idx.sort(-1).values[:, 0].type(torch.int)
    return idx

In [116]:
with open('data.pkl', 'rb') as handle:
    data = pickle.load(handle)

grid_size = data["grid_size"]
window_offset = data["window_offset"]
window_size = data["window_size"]

obj_points = torch.tensor(data["obj_points"])
obj_normals = torch.tensor(data["obj_normals"])

env_points = torch.tensor(data["env_points"])
sun_vectors = torch.tensor(data["sun_vecs"])

In [117]:
window_inset_angle = torch.atan(torch.tensor(2 * window_offset / window_size))


obj_pts = (obj_points[:, None, :] * torch.ones_like(sun_vectors)[None, :, :]).view(-1, 3)
obj_nrm = (obj_normals[:, None, :] * torch.ones_like(sun_vectors)[None, :, :]).view(-1, 3)

sun_vecs = (torch.ones_like(obj_points)[:, None, :]  * sun_vectors[None, :, :]).view(-1, 3)

In [118]:
angles = ((obj_nrm / torch.linalg.vector_norm(obj_nrm, dim=-1, keepdim=True))[:, None, :] @ (sun_vecs / torch.linalg.vector_norm(sun_vecs, dim=-1, keepdim=True))[:, :, None]).view(-1).acos()
angles_mask = angles > ((0.5 * torch.pi) + window_inset_angle)

obj_pts = obj_pts[angles_mask]
obj_nrm = obj_nrm[angles_mask]
sun_vecs = sun_vecs[angles_mask]
indices = torch.arange(angles_mask.shape[0])[angles_mask]

In [138]:
shift = first_non_zero(obj_nrm)
print(obj_nrm.shape)
print(shift.shape)
obj_nrm = obj_nrm.roll(tuple(shift.tolist()), 1)

print(obj_nrm[:10])

torch.Size([2201, 3])
torch.Size([2201])


RuntimeError: shifts and dimensions must align. shifts: 2201, dims:1